In [1]:
import numpy as np

ALL_WORDS = np.array(open('../data/scrabble_dict.txt', 'r').read().split('\n'))

def get_valid_words(puzzle):
    """
    Return valid words given puzzle letters, with first and last letter arrays.
    """
    sideconv = np.vectorize({**{j: int(puzzle.index(j)/3)+1 if j in puzzle else -1
                                for j in [chr(i+65) for i in range(26)]},
                             '': 0}.get)
    word_letters = ALL_WORDS.view('<U1').reshape(ALL_WORDS.shape[0], -1)
    word_sides = sideconv(word_letters)
    
    filt = (~np.any(word_sides == -1, axis=1))\
           & (~np.any((word_sides[:, 1:] > 0) 
                      & (word_sides[:, 1:] == word_sides[:, :-1]), axis=1))

    return (ALL_WORDS[filt],
            word_letters[filt, 0],
            word_letters[filt][np.arange(np.sum(filt)), 
                               np.sum(word_letters[filt] != '', axis=1)-1])

def solve(puzzle):
    """
    Find all solutions to puzzle with shortest possible number of words.
    """
    valid_words, first_letter, last_letter = get_valid_words(puzzle)
    
    poss = np.expand_dims(valid_words, 1)
    poss_ll = np.copy(last_letter)

    while True:
        ulchk = poss.view('<U1').reshape(poss.shape[0], -1)
        used_letters = np.column_stack([np.any(ulchk == i, axis=1) for i in puzzle])
        n_used_letters = np.sum(used_letters, axis=1)
        if np.sum(n_used_letters == 12):
            break

        lmfilt = np.repeat(poss_ll, valid_words.shape[0]) == np.tile(first_letter, poss.shape[0])
        ix1 = np.repeat(np.arange(poss.shape[0]), valid_words.shape[0])[lmfilt]
        ix2 = np.tile(np.arange(valid_words.shape[0]), poss.shape[0])[lmfilt]
        poss = np.column_stack((poss[ix1], valid_words[ix2]))
        poss_ll = last_letter[ix2]

    return poss[n_used_letters == 12]

In [2]:
solve('KHMBUOEGICTN')

array([['BEHEMOTH', 'HUCKING'],
       ['BEMOCKING', 'GUICHET'],
       ['BETHUMB', 'BECKONING'],
       ['BETHUMB', 'BEMOCKING'],
       ['BIOMIMETIC', 'CHUCKING'],
       ['BIOMIMETIC', 'CHUNKING'],
       ['COCKET', 'THUMBING'],
       ['KOHUTUHUTU', 'UNBECOMING'],
       ['NOCKET', 'THUMBING'],
       ['TECHNIKON', 'NUMBING'],
       ['THUMB', 'BECKONING'],
       ['THUMB', 'BEMOCKING'],
       ['THUMBKIN', 'NEOGOTHIC']], dtype='<U15')